In [ ]:
import os
import numpy, astropy
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.pyplot as plt
from photutils.aperture import SkyEllipticalAperture, EllipticalAperture, SkyRectangularAperture, RectangularAperture 

This notebook takes an ALFALFA cube and extract a 1-dimensional spectrum at a specified location. It does this first over an area equal to the beam shape and then over a larger square region.

**Note**: You will need the ALFALFA 1044+13 grid in that data directory in order for this notebook to run successfully. You can obtain this by running the [download_example_data.py](../scripts/download_example_data.py) script in the scripts directory. If you wish to run this with other grids then you can find instructions for accessing the grids in the [grid_access.md](../docs/grid_access.md) file in the docs folder or illustrated instructions on the [wiki](https://github.com/jonesmg/ALFALFA_Legacy/wiki/Grid-access-via-NRAO-archive).

In [ ]:
#Define the path to the data
#You should already have run the data download script
cwd = os.getcwd()+'/'
data_path = cwd+'../data/A2010/pipeline.unknown_date/'

In [ ]:
#Define the grid you are using
grid_ra = '1044'
grid_dec = '13'
freq_slice = 'a'

grid_filename = f'{grid_ra}+{grid_dec}{freq_slice}_spectral.fits'

#Open cube FITS files
hdu = fits.open(data_path+grid_filename)

#Extract the data from fits file and average together the two polarizations
cube = numpy.mean(hdu[0].data,axis=0)

#Make header
header = hdu[0].header

#Extract beam parameters
bmaj, bmin, bpa = header['BMAJ']*u.deg, header['BMIN']*u.deg, 0*u.deg
pixsize = header['CDELT2']*u.deg

#Make the beam correction factor
beam_factor = (numpy.pi*bmaj*bmin/(pixsize**2.))/(4.*numpy.log(2.))

#Extract the channel width from the header and build frequency array
chan_df = header['CDELT3']*u.MHz
freq = (header['CRVAL3'] + header['CDELT3']*numpy.arange(0,1024,1))*u.MHz

#Now build velocity array
rest_freq = hdu[0].header['RESTFREQ']*u.MHz/1E6
vel = (astropy.constants.c.value*u.km/u.s/1000)*(rest_freq-freq)/freq

#Make World Coordinate System
grid_wcs = WCS(header,naxis=2)

In [ ]:
#Start by extracting a spectrum in the shape of the beam

#Define a position to extract a spectrum at
#In this case we will use NGC3338
spec_pos = SkyCoord(160.531332, 13.747050, unit=u.deg)

#Make an aperture that's the same shape as the beam
sky_aperture = SkyEllipticalAperture(spec_pos, a=bmaj/2, b=bmin/2, theta=bpa)

#Convert to a pixel aperture
pix_aperture = sky_aperture.to_pixel(grid_wcs)

In [ ]:
#Plot the aperture on the grid

fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=grid_wcs)
plt.imshow(cube[512], origin='lower', cmap='Greys', aspect='equal')

pix_aperture.plot(color='orange',lw=1.5,ls='-')

plt.xlabel(r'RA')
plt.ylabel(r'Dec')

In [ ]:
#Now extract the spectrum
pix_aperture_mask = pix_aperture.to_mask(method='exact')

#Create empty spectrum
spec = numpy.zeros(len(freq))

#Step through cube channels and extract flux in each
for i in range(len(spec)):
    spec[i] = numpy.sum(pix_aperture_mask.multiply(cube[i]))/beam_factor

#Set units
spec = spec*u.mJy

In [ ]:
plt.plot(vel,spec)

plt.axhline(0.,c='k',lw=1)
plt.ylim(-10.,200.)

plt.xlabel('Velocity [km/s] (Heliocentric)')
plt.ylabel('Flux Density [mJy]')

The spectral profile looks quite peculiar, this is likely because the source extends slightly beyond one beam and we are therefore not including all the emission.</br></br> Now let's switch to using a large 20'x20' square extraction region.

In [ ]:
#Make an aperture that's the same shape as the beam
sky_aperture = SkyRectangularAperture(spec_pos,w=20*u.arcmin,h=20*u.arcmin)

#Convert to a pixel aperture
pix_aperture = sky_aperture.to_pixel(grid_wcs)

In [ ]:
#Plot the aperture on the grid

fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=grid_wcs)
plt.imshow(cube[512], origin='lower', cmap='Greys', aspect='equal')

pix_aperture.plot(color='orange',lw=1.5,ls='-')

plt.xlabel(r'RA')
plt.ylabel(r'Dec')

In [ ]:
#Now extract the spectrum
pix_aperture_mask = pix_aperture.to_mask(method='exact')

#Create empty spectrum
spec = numpy.zeros(len(freq))

#Step through cube channels and extract flux in each
for i in range(len(spec)):
    spec[i] = numpy.sum(pix_aperture_mask.multiply(cube[i]))/beam_factor

#Set units
spec = spec*u.mJy

In [ ]:
plt.plot(vel,spec)

plt.axhline(0.,c='k',lw=1)
plt.ylim(-50.,1200.)

plt.xlabel('Velocity [km/s] (Heliocentric)')
plt.ylabel('Flux Density [mJy]')